### Section bodies combined by position order top 1000 words

#### Steps:

1. Get the relevant sections and their position in the report
2. Order the sections by position
3. Extract the body of section and keep on concatenating
4. Extract top 1000 words
5. If no relevant sections -> Write the complete file as summary with top 1000 words
6. Compare the system summary with each gold summary and get the rouge score. Take average of all gold summaries for the respective file.
7. Take average of all the scores for all the files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

In [ ]:
pip install rouge_score

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
import torch.nn.functional as F

In [ ]:
import pandas as pd
import pickle
import os
from tqdm import tqdm
from extract_section_body import extract_section_body
from rouge_evaluation import get_rouge_scores

In [ ]:
model_path='/content/drive/MyDrive/NLP_Project/fine_tuned_model'
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
VALIDATION_DATASET = True
TEST_DATASET = False

In [ ]:
if VALIDATION_DATASET:
    dir_ = '/content/drive/MyDrive/FNS_Dataset_2023/validation/'
    toc_loc_pkl_file_path = '/content/drive/MyDrive/FNS_Dataset_2023/validation/out/valid_toc_loc.pkl'
    df_predicted_path = '/content/drive/MyDrive/NLP_Project/2_Section_Classification/out/validation_df_predicted.pkl'

if TEST_DATASET:
    dir_ = '/content/drive/MyDrive/FNS_Dataset_2023/testing/'
    toc_loc_pkl_file_path = '/content/drive/MyDrive/FNS_Dataset_2023/testing/out/test_toc_loc.pkl'
    df_predicted_path = '/content/drive/MyDrive/NLP_Project/2_Section_Classification/out/test_df_predicted.pkl'

annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
system_summary_dir = 'section_combined_by_position_BART'
team_name = 'IT-356'
dir_

'/content/drive/MyDrive/FNS_Dataset_2023/validation/'

In [ ]:
df_predicted = pickle.load(open(df_predicted_path, 'rb'))
df_predicted

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary,toc_section_cleaned,pred,False,True
0,30777,Financial and operational highlights,161,22,0,financi oper highlight,1,0.407563,0.592437
1,30777,Strategic report,183,6,1,strateg report,0,0.932972,0.067028
2,30777,Global network,189,11,0,global network,0,0.756613,0.243387
3,30777,Chairman’s statement,200,4,1,chairman statement,1,0.018573,0.981427
4,30777,Chief Executive’s review,204,4,1,chief execut review,1,0.007140,0.992860
...,...,...,...,...,...,...,...,...,...
10547,4162,"to 110,",21914,1104,0,,0,0.983343,0.016657
10548,4162,and 117,23018,1104,0,,0,0.985658,0.014342
10549,4162,to 116,24122,1104,0,,0,0.987655,0.012345
10550,4162,to 122,25226,5440,0,,0,0.987178,0.012822


In [ ]:
def get_relevant_sections_with_score(file_id):
    result_dict = df_predicted[(df_predicted.file_id == int(file_id)) & (df_predicted.pred == 1)][['toc_section', 'True']].to_dict('list')
    return result_dict

In [ ]:
import shutil
shutil.rmtree(system_summary_dir)

In [ ]:
def summarize(text):
  inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
    summary_ids = fine_tuned_model.generate(inputs['input_ids'].to(device),num_beams=2)

  # Decode the generated summary
  generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return generated_summary

In [ ]:
os.makedirs(system_summary_dir)
#large_files=['4115','4117','31014']
total_files = len(os.listdir(os.path.join(dir_, annual_reports_dir)))
for file in tqdm(os.listdir(os.path.join(dir_, annual_reports_dir)), desc="Processing Files", total=total_files, ncols=80, colour='green', leave=True):
    try:
        file_id = file.split('.')[0]
        relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        # Section order is maintained
        relevant_sections = relevant_sections_with_score['toc_section']
        #print(file_id, relevant_sections)
        summary = ""
        if relevant_sections:
            #print('Relevant Section Found in ', file_id)
            for section in relevant_sections:
                summary = summary + " " + extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            #print(f"Number of words in narrative sections of {file_id}: ",number_of_words)
            if number_of_words > 20000:
              summary = " ".join(summary_split[:1000])
            elif number_of_words > 1000:
                summary = summarize(summary)#" ".join(summary_split[:1000])
                summary_split = summary.split(' ')
                summary = " ".join(summary_split[:1000])

        else:
            #print('Relevant Section Not Found in ', file_id)
            summary = open(os.path.join(dir_, annual_reports_dir, file), "r", encoding="utf-8").read()
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            #print(number_of_words)
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
        summary_split = summary.split(' ')
        number_of_words = len(summary_split)
        print(number_of_words)
        with open(os.path.join(system_summary_dir, file_id+'_'+team_name+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))

        if ".DS_Store" in file:
            continue
    except Exception as e:
        print(file, e)

Processing Files:   0%|                       | 1/413 [00:46<5:21:20, 46.80s/it]

119


Processing Files:   0%|                       | 2/413 [01:31<5:12:34, 45.63s/it]

88
1000


Processing Files:   1%|▏                      | 4/413 [02:16<3:31:39, 31.05s/it]

...

104


Processing Files: 100%|████████████████████▉| 411/413 [4:29:49<01:13, 36.55s/it]

89


Processing Files: 100%|█████████████████████| 413/413 [4:30:31<00:00, 39.30s/it]

118
1000


In [ ]:
if VALIDATION_DATASET:
    gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
    rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
    print(rouge_scores)

Processing File Number:  0
Processing File Number:  50
Processing File Number:  100
Processing File Number:  150
Processing File Number:  200
Processing File Number:  250
Processing File Number:  300
Processing File Number:  350
Processing File Number:  400
Number of files processed:  413
{'rouge-1': {'p': 0.6602882333238651, 'r': 0.11734237359160422, 'f': 0.14872965358895418}, 'rouge-2': {'p': 0.343617364418775, 'r': 0.06763684738519223, 'f': 0.07873979202990374}}


In [ ]:
rouge_scores

{'rouge-1': {'p': 0.6602882333238651,
  'r': 0.11734237359160422,
  'f': 0.14872965358895418},
 'rouge-2': {'p': 0.343617364418775,
  'r': 0.06763684738519223,
  'f': 0.07873979202990374}}

In [ ]:
import shutil
source_folder = '/content/section_combined_by_position_BART'
destination_folder = '/content/drive/MyDrive/NLP_Project/section_combined_by_position_BART'

shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/NLP_Project/section_combined_by_position_BART'